In [3]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Scrape Latest Customer Reviews - Amazon Product

## Provide any product id as asin

In [ ]:
# Scrape the latest reviews for product with given ASIN using scraperapi
asin = 'B08N5NQ869'
soup_contents = []
for page in range(1,60):
  time.sleep(20) # avoids CAPTCHA
  url = f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page}'
  payload = {'api_key':'bd6c22692874b6bd87ff88737c5f3d6e',
          'url': url}
  response = requests.get('http://api.scraperapi.com', params = payload)
  soup = BeautifulSoup(response.content, 'html.parser')
  soup_contents.append(soup)

In [ ]:
# Check content of the html
soup = soup_contents[2]
soup.prettify

In [ ]:
ratings = soup.find_all('div', attrs={'class' : "a-section celwidget"})
rating = ratings[0]

title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
date = rating.find('span', attrs = {'data-hook':'review-date'}).text
try:
  verified = rating.find('span', attrs = {'class':'avp-badge'}).text
except:
  verified = 'Not verified'
star = float(rating.find('a', attrs = {'class':'a-link-normal'}).text[:3])
try:
  votes = rating.find('span', attrs={'class':'review-votes'}).text
except:
  votes = 'no vote'

prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
color = prod[1].split('Config')[0].strip()
configuration = prod[-1].strip()

print(title)
print(body)
print(date)
print(verified)
print(star)
print(votes)
print(color)
print(configuration)

Love it.
Exactly what we were looking for.
Reviewed in the United States on April 26, 2023
Not verified
5.0
no vote
Venetian Bronze
Doorbell only


In [ ]:
reviews_list = []
for page in soup_contents:
  # Extrack all user reviews for the page
  ratings = page.find_all('div', attrs={'class' : "a-section celwidget"})

  # Iterate over each review and extract rewiew contents
  for rating in ratings:
    title = rating.find('a', attrs = {'data-hook':'review-title'}).text.strip('\n')
    body = rating.find('span', attrs = {'data-hook':'review-body'}).text.strip('\n')
    date = rating.find('span', attrs = {'data-hook':'review-date'}).text
    try:
      verified = rating.find('span', attrs = {'data-hook':'avp-badge'}).text
    except:
      verified = 'Not verified'
    star = rating.find('a', attrs = {'class':'a-link-normal'}).text
    try:
      votes = rating.find('span', attrs = {'data-hook':'helpful-vote-statement'}).text
    except:
      votes = 'no vote'
    if rating.find('img').get('src'):
      img = 'Yes'
    else:
      img = 'No'

    prod = rating.find('a', attrs = {'data-hook':'format-strip'}).text.split(':')
    color = prod[1].split('Config')[0].strip()
    configuration = prod[-1].strip()

    # Create a dictionary for the review
    rating_dict = {'title': title,
                  'body': body,
                  'date': date,
                  'status': verified,
                  'votes': votes,
                  'contains_image': img,
                  'color':color,
                  'configuration':configuration,
                  'score': star,
                  }

    # Append the dictionary review object to the list
    reviews_list.append(rating_dict)


In [ ]:
# Convert into dataframe
data = pd.DataFrame(reviews_list)
data['product'] = 'Ring Video Doorbell - 1080p HD video, improved motion detection, easy installation'
data['asin'] = 'B08N5NQ869'
# Save Data
data.to_csv('ring_video_doorbell.csv', index = False)
# Load and check data
data = pd.read_csv('ring_video_doorbell.csv')
data.head()

,title,body,date,status,votes,contains_image,color,configuration,score,product,asin
0,Love it.,Exactly what we were looking for.,"Reviewed in the United States on April 26, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,5.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
1,Son feels safer,Easy to install!,"Reviewed in the United States on April 26, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,5.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
2,Great Device - Beware of Pre-Setup,I have for Amazon Echo Dot's in my home and pu...,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,4.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
3,Not bad,Should have gotten the battery one.,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Venetian Bronze,Doorbell only,3.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869
4,Door ring,Must keep charging batteries,"Reviewed in the United States on April 25, 2023",Verified Purchase,no vote,Yes,Satin Nickel,Doorbell only,2.0 out of 5 stars,"Ring Video Doorbell - 1080p HD video, improved...",B08N5NQ869


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           520 non-null    object
 1   body            520 non-null    object
 2   date            520 non-null    object
 3   status          520 non-null    object
 4   votes           520 non-null    object
 5   contains_image  520 non-null    object
 6   color           520 non-null    object
 7   configuration   520 non-null    object
 8   score           520 non-null    object
 9   product         520 non-null    object
 10  asin            520 non-null    object
dtypes: object(11)
memory usage: 44.8+ KB


In [ ]:
data['rating'] = data['score'].str[:3]

In [ ]:
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

In [ ]:
data = data.reindex(columns=['title', 'rating', 'body'])

In [ ]:
data['title'] = data['title'].str.split('\n').str[1]

In [ ]:
data['text'] = data['title'] + ' ' + data['body']
data.head()

,title,rating,body,text
0,Ring camera,1.0,I like the camera but it skips time and I hate...,Ring camera I like the camera but it skips tim...
1,Missing pieces,3.0,Note: I will come back and modify my review i...,Missing pieces Note: I will come back and mod...
2,Just what I was looking for,5.0,Love my ring camera. Easy to install and use. ...,Just what I was looking for Love my ring camer...
3,Love this thing,5.0,Wish I would have bought one of these years ag...,Love this thing Wish I would have bought one o...
4,Clear views!,5.0,Video is clear and accessible! Battery lasts a...,Clear views! Video is clear and accessible! Ba...


In [4]:
# Save Data
data.to_csv('amazon-review-preprocessesd-data.csv', index = False)
# Load and check data
data = pd.read_csv('amazon-review-preprocessesd-data.csv')
data.head()

,title,rating,body,text
0,Sound not working,1.0,After a few months you can hear when someone i...,Sound not working After a few months you can h...
1,Carma,5.0,Works ok skips some of the video some times,Carma Works ok skips some of the video some times
2,Just okay,2.0,"The video quality is great, night vision is go...","Just okay The video quality is great, night vi..."
3,You have to purchase video capture separately,1.0,After your free trial is over for motion detec...,You have to purchase video capture separately ...
4,Awesome camera,5.0,Great camera to watch over the inside of the h...,Awesome camera Great camera to watch over the ...


# Pre-Trained Transformer: bert-base-multilingual-uncased-sentiment
This a bert-base-multilingual-uncased model finetuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian. It predicts the sentiment of the review as a number of stars (between 1 and 5).

This model is intended for direct use as a sentiment analysis model for product reviews in any of the six languages above, or for further finetuning on related sentiment analysis tasks.

In [ ]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


In [ ]:
# import modules
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
def sentiment_score(review):
    """ Tokenize review and predict sentiment class """
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

def sentiment_confidence(review):
    """ Tokenize review and return sentiment class probability for the most likely class """
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    logits=result.logits.detach().numpy()
    logits=np.exp(logits)/np.sum(np.exp(logits))
    ind=int(np.argmax(logits))
    return logits[0][ind]

In [ ]:
data['sentiment'] = data['text'].apply(lambda x: sentiment_score(x[:512]))
data['sent_confidence'] = data['text'].apply(lambda x: sentiment_confidence(x[:512]))

In [ ]:
data[['title','rating','sentiment','sent_confidence']].sort_values(by = 'sent_confidence', ascending = False).head(20)

**Evaluation Metrics**

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Assuming your dataframe is named 'data'
data = data.dropna(subset=['rating', 'sentiment'])
confusion_matrix(data.rating.values, data.sentiment.values)


In [ ]:
acc_exact = sum(data['rating'] == data['sentiment'])/len(data)
acc_by_1 = sum(np.abs(data['rating'] - data['sentiment']) <= 1)/len(data)
print(f'Accuracy (exact)  is {round(acc_exact, 2)}')
print(f'Accuracy (off-by-1) is {round(acc_by_1, 2)}')

In [ ]:
# personal review
def predict_my_rating():
  review = input('Insert your review : ')
  print(f"Your rating for the product is {sentiment_score(review)}")

predict_my_rating()

#Deep Learning Approach
##Convolutional Neural Network (CNN) model for text classification

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Sample data
reviews = data['text'].tolist()

labels = ['good product', 'worst product']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

# Pad sequences to have the same length
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'good product': 1, 'worst product': 0}
numeric_labels = np.array([label_mapping[label] for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, numeric_labels, test_size=0.2, random_state=42)

# Build the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=16, validation_data=(np.array(X_test), np.array(y_test)))

# Make predictions
new_reviews = ['Excellent product!', 'Awful experience.']
new_sequences = tokenizer.texts_to_sequences(new_reviews)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length)
predictions = model.predict(np.array(new_padded_sequences))

for i, review in enumerate(new_reviews):
    if predictions[i] > 0.5:
        print(f'Review: {review} - Category: good product')
    else:
        print(f'Review: {review} - Category: worst product')


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6913 - accuracy: 0.6667 - val_loss: 0.6687 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 45ms/step - loss: 0.6567 - accuracy: 0.6667 - val_loss: 0.6721 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 59ms/step - loss: 0.6268 - accuracy: 1.0000 - val_loss: 0.6762 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 64ms/step - loss: 0.6006 - accuracy: 1.0000 - val_loss: 0.6812 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 45ms/step - loss: 0.5780 - accuracy: 1.0000 - val_loss: 0.6854 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 64ms/step - loss: 0.5574 - accuracy: 1.0000 - val_loss: 0.6887 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 62ms/step - loss: 0.5380 - accuracy: 1.0000 - val_loss: 0.6916 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=============

In [ ]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Deep Learning Approach
##LSTM - Long Short Term Memory

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Sample data
reviews = data['text'].tolist()
labels = ['good product', 'worst product']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

# Pad the sequences to have the same length
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Convert labels to numerical values
label_mapping = {'good product': 1, 'worst product': 0}
numeric_labels = np.array([label_mapping[label] for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, numeric_labels, test_size=0.2, random_state=42)

# Build the LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Make predictions
new_reviews = ['Excellent product!', 'Awful experience.']
new_sequences = tokenizer.texts_to_sequences(new_reviews)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length)
predictions = model.predict(new_padded_sequences)

for i, review in enumerate(new_reviews):
    if predictions[i] > 0.5:
        print(f'Review: {review} - Category: good product')
    else:
        print(f'Review: {review} - Category: worst product')


In [ ]:
model.save('model.h5')